In [98]:
%load_ext autoreload
%autoreload 2

import os 
import xarray as xr
import torch
import dask
import pandas as pd
from torch.utils.data import DataLoader
import numpy as np
import logging

from aurora import Aurora, AuroraSmall, rollout

from aurora_benchmark.utils import verbose_print, xr_to_netcdf

dask.config.set(scheduler='threads')

from aurora_benchmark.data import (
    XRAuroraDataset, 
    aurora_batch_collate_fn, 
    aurora_batch_to_xr, 
    unpack_aurora_batch
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
p = "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.nc"
ph5 = "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.h5"
pzarr = "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr"

d = xr.open_dataset(p, engine="netcdf4")
d.to_zarr(pzarr)

In [101]:
era5_surface_paths = [
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
]
era5_atmospheric_paths = [ # we will repeat on level dimension
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
]
era5_static_paths = [ # we will select the first time step
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
    "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.zarr",
]

surf_vars = ["2t", "msl", "10u"]
atmospheric_vars = ["t", "q", "u"]
static_vars = ["z", "lsm", "slt"]

era5_surface_paths_h5, era5_atmospheric_paths_h5, era5_static_paths_h5 = [], [], []
for f in era5_surface_paths:
    ff = f.replace(".nc", ".h5netcdf.nc")
    xr.to_netcdf(xr.open_dataset(f, engine="netcdf4"), f, engine="h5netcdf")
    era5_static_paths_h5.append(ff)
for f in era5_atmospheric_paths:
    ff = f.replace(".nc", ".h5netcdf.nc")
    xr.to_netcdf(xr.open_dataset(f, engine="netcdf4"), f, engine="h5netcdf")
    era5_static_paths_h5.append(ff)
for f in era5_static_paths:
    ff = f.replace(".nc", ".h5netcdf.nc")
    xr.to_netcdf(xr.open_dataset(f, engine="netcdf4"), f, engine="h5netcdf")
    era5_static_paths_h5.append(ff)

# Load the data into a single dataset with the same coords but multiple variables
surface_dss = [
    xr.open_dataset(path, engine="h5netcdf").rename({"msl": svar}).drop_vars("time_bnds")
    for path, svar in zip(era5_surface_paths_h5, surf_vars)
]
surface_ds = xr.merge(surface_dss)
atmospheric_dss = [
    xr.open_dataset(path, engine="h5netcdf").rename({"msl": svar}).expand_dims({"level": [1000, 700, 250]}).drop_vars("time_bnds")
    for path, svar in zip(era5_atmospheric_paths_h5, atmospheric_vars)
]
atmospheric_ds = xr.merge(atmospheric_dss)
static_dss = [
    xr.open_dataset(path, engine="h5netcdf").rename({"msl": svar}).isel(time=0).drop_vars("time_bnds")
    for path, svar in zip(era5_static_paths_h5, static_vars)
]
static_ds = xr.merge(static_dss)

# rename coord lat to latitude and lon to longitude
surface_ds = surface_ds.rename({"lat": "latitude", "lon": "longitude"})
atmospheric_ds = atmospheric_ds.rename({"lat": "latitude", "lon": "longitude"})
static_ds = static_ds.rename({"lat": "latitude", "lon": "longitude"})

surface_ds.dims, atmospheric_ds.dims, static_ds.dims

OSError: Unable to synchronously open file (file signature not found)

In [95]:
batch_size = 8
num_workers = 2
eval_start ="7d"
era5_base_frequency = "1d"
forecast_horizon = "5w"

warmup_steps = int(pd.Timedelta(eval_start) / pd.Timedelta(era5_base_frequency)) if eval_start is not None else 0
forecast_steps = int(pd.Timedelta(forecast_horizon) / pd.Timedelta(era5_base_frequency))

dataset = XRAuroraDataset(
    surface_ds=surface_ds,
    atmospheric_ds=atmospheric_ds,
    static_ds=static_ds,
    init_frequency="1d",
    forecast_horizon=forecast_horizon,
    num_time_samples=2, # Aurora has fixed history length of 2...
    drop_timestamps=True,
    persist=False,
    rechunk=False,
    atmospheric_variables=atmospheric_vars,
    surface_variables=surf_vars,
    static_variables=static_vars,
)

eval_loader = DataLoader(
    dataset, 
    batch_size=batch_size, 
    collate_fn=aurora_batch_collate_fn,
    num_workers=num_workers
)

/gpfs/home3/ewalt/aurora-benchmark/aurora_benchmark/data.py:306: UserWarning: The init_frequency is not 1 day. This has not been tested.
  raise NotImplementedError("Replacement variables are not yet implemented.")


In [96]:
AURORA_VARIABLE_RENAMES = {
    "surface": {
        "u10": "10u",
        "v10": "10v",
        "t2m": "2t",
    },
    "atmospheric": {},
    "static": {},
}
INVERTED_AURORA_VARIABLE_RENAMES = {
    "surface": {v: k for k, v in AURORA_VARIABLE_RENAMES["surface"].items()},
    "atmospheric": {v: k for k, v in AURORA_VARIABLE_RENAMES["atmospheric"].items()},
    "static": {v: k for k, v in AURORA_VARIABLE_RENAMES["static"].items()},
}

interest_variables = atmospheric_vars + surf_vars
interest_levels = [1000, 700, 250]


verbose = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("loading model ...")
# aurora_model = "aurora-0.25-pretrained.ckpt"
# model = Aurora(use_lora=False)
# model.load_checkpoint("microsoft/aurora", aurora_model)
# model = model.to(device)

print(f"Evaluating on {device}")
# evaluation loop
xr_preds = {"surface_ds": [], "atmospheric_ds": []}
with torch.inference_mode() and torch.no_grad():
    for i, batch in enumerate(eval_loader):
        batch = batch.to(device)
        # rollout until for forecast_steps
        print(f"Rollout prediction on batch {i} ...")
        trajectories = [[] for _ in range(batch_size)]
        for s, batch_pred in enumerate(rollout(model, batch, steps=forecast_steps)):
            if s < warmup_steps:
                print(f" * Rollout step {s+1}: skipping warmup period")
                continue            
            # separate batched batches
            sub_batch_preds = unpack_aurora_batch(batch_pred.to("cpu"))
            print(f" * Rollout step {s+1}: unpacked {len(sub_batch_preds)} sub-batches")
            assert len(sub_batch_preds) == batch_size
            # accumulate
            for b, sub_batch_pred in enumerate(sub_batch_preds):
                trajectories[b].append(sub_batch_pred)
        print(f"Processing trajectories ...")
        # convert to xr 
        for init_time, trajectory in zip(batch.metadata.time, trajectories):
            print(f" * init_time={init_time}: combining {len(trajectory)} steps")
            assert len(trajectory) == forecast_steps-warmup_steps
            # collate trajectory batches
            trajectory = aurora_batch_collate_fn(trajectory)
            # convert to xr.Dataset
            trajectory = aurora_batch_to_xr(trajectory, frequency=era5_base_frequency)
            # add lead time
            for var_type, vars_ds in trajectory.items():
                # ensure processing is necessary
                if var_type == "static_ds":
                    print(f" * Skipping static variables")
                    continue # we do not care about static variables for the forecast
                if not any([var in vars_ds.data_vars for var in interest_variables]):
                    print(f" * Skipping {var_type} variables as no interest variables are present")
                    continue # don't bother processing variables we are not interested in
                if var_type == "atmospheric_ds" and (interest_levels is None or len(interest_levels)==0):
                    print(f" * Skipping atmospheric variables as no interest levels have been requested")
                    continue # we do not care about atmospheric variables if no levels are of interest
                
                # select interest variables and levels
                vars_interest_variables = [var for var in vars_ds.data_vars if var in interest_variables]
                if var_type == "atmospheric_ds":
                    vars_ds = vars_ds[vars_interest_variables].sel(level=interest_levels)
                else:
                    vars_ds = vars_ds[vars_interest_variables]
                
                # add lead time
                vars_ds = vars_ds.assign_coords({"lead_time": vars_ds.time.values - np.datetime64(init_time)})
                vars_ds = vars_ds.set_index({"lead_time": "lead_time"})
                
                # TODO: aggregate desired timesteps to agg freq  
                # vars_ds = vars_ds.resample(time=eval_aggregation).mean()  # not enough as it messes with "lead time"                    
                
                # append to predictions
                xr_preds[var_type].append(vars_ds)
        
# merge predictions and save
for var_type, var_ds_list in xr_preds.items():
    ds = xr.concat(var_ds_list, dim="time").rename(INVERTED_AURORA_VARIABLE_RENAMES[var_type])

loading model ...
Evaluating on cuda
Rollout prediction on batch 0 ...
 * Rollout step 1: skipping warmup period
 * Rollout step 2: skipping warmup period
 * Rollout step 3: skipping warmup period
 * Rollout step 4: skipping warmup period
 * Rollout step 5: skipping warmup period
 * Rollout step 6: skipping warmup period
 * Rollout step 7: skipping warmup period
 * Rollout step 8: unpacked 8 sub-batches
 * Rollout step 9: unpacked 8 sub-batches
 * Rollout step 10: unpacked 8 sub-batches
 * Rollout step 11: unpacked 8 sub-batches
 * Rollout step 12: unpacked 8 sub-batches
 * Rollout step 13: unpacked 8 sub-batches
 * Rollout step 14: unpacked 8 sub-batches
 * Rollout step 15: unpacked 8 sub-batches
 * Rollout step 16: unpacked 8 sub-batches
 * Rollout step 17: unpacked 8 sub-batches
 * Rollout step 18: unpacked 8 sub-batches
 * Rollout step 19: unpacked 8 sub-batches
 * Rollout step 20: unpacked 8 sub-batches
 * Rollout step 21: unpacked 8 sub-batches
 * Rollout step 22: unpacked 8 sub-

KeyboardInterrupt: 